In [187]:
def euclidean(ai, bi):
    return ((ai - bi))**2

In [188]:
def calcDistance(query, instance, weighted):
    distance = 0
    weight = 1
    for i in range(len(instance)-1):
        distance = distance + euclidean(query[i],instance[i])
    distance = (distance)**0.5
    if (weighted == 1):
        weight = 1/(distance**2)
    return ((distance, weight))

In [186]:
def vdm(ai, bi, attributeCount, pos, classList):
    nia = 0
    nib = 0
    vdmResult = 0
    for i in range(len(attributeCount)):
        if (attributeCount[i][0] == ai and attributeCount[i][2] == pos):
            nia = nia + attributeCount[i][3]
        if (attributeCount[i][0] == bi and attributeCount[i][2] == pos):
            nib = nib + attributeCount[i][3]
                   
    for j in range(len(classList)):
        niac = 0
        nibc = 0
        for i in range(len(attributeCount)):
            if (attributeCount[i][0] == ai and attributeCount[i][1] == classList[j] and attributeCount[i][2] == pos):
                niac = niac + attributeCount[i][3]
            if (attributeCount[i][0] == ai and attributeCount[i][1] == classList[j] and attributeCount[i][2] == pos):
                nibc = nibc + attributeCount[i][3]
        
        if (nia == 0):
            v1 = 0
        else:
            v1 = niac/nia
        
        if (nib == 0):
            v2 = 0
        else:
            v2 = nibc/nib
        
        calc = abs(v1-v2)
        vdmResult = vdmResult + calc
        
    return vdmResult

In [185]:
def calcVDM(query, instance, weighted, attributeCount, classList):
    distance = 0
    weight = 1
    for i in range(len(query)):
        distance = distance + vdm(query[i], instance[i], attributeCount, i, classList)
    
    distance = (distance)**0.5
    
    if (weighted == 1):
        weight = 1/(distance**2)
    
    return ((distance, weight))

In [198]:
def calcHVDM(query, instance, weighted, attributeCount, classList, typeList):
    distance = 0
    weight = 1
    for i in range(len(query)):
        if (typeList[i] == 'num'):
            distance = distance + ((euclidean(query[i], instance[i]))**2)
        elif (typeList[i] == 'cat'):
            distance = distance + ((vdm(query[i], instance[i], attributeCount, i, classList))**2)
    
    distance = (distance)**0.5
    
    if (weighted == 1):
        weight = 1/(distance**2)
    
    return ((distance, weight))

In [190]:
def knn (query, n, data, weighted, dataType, typeList):
    # k = testInstance
    # 
    #dataType 0 = numérico, 1 = categorico, 2 = heterogeneo
    distances = []
    classes = []
    
    attributeCount = []
    
    classColumn = [row[-1] for row in data]
    classList = []
    
    for i in range(len(classColumn)):
        if (classColumn[i] not in classList):
            classList.append(classColumn[i])
    
    for i in range(len(data[0])-1):
        for j in range(len(data)):
            flag = 0
            for k in range(len(attributeCount)):
                if (data[j][i]==attributeCount[k][0] and data[j][-1]==attributeCount[k][1] and i == attributeCount[k][2]):
                    attributeCount[k][3] = attributeCount[k][3] + 1
                    flag = 1
                
            if(flag == 0):
                attributeCount.append([data[j][i], data[j][-1], i, 1])
            
            #if (data[j][i] not in attr):
             #   if (len(attr)==0):
              #      attributeCount.append([data[j][i], data[j][-1], 1])
               # else:
                #    for k in range(len(attributeCount)):
                 #       if (data[j][i]==attributeCount[k][0] and data[j][-1]==attributeCount[k][1]):
                  #          attributeCount[k][2] = attributeCount[k][2] + 1
                #attr.append([data[j][i], data[j][-1]])

    #euclidean
    if (dataType == 0):
        for i in range(len(data)):
            distance = calcDistance(query, data[i], weighted)
            distances.append((data[i], distance[0], distance[1]))
        distances.sort(key=lambda x: x[0], reverse=True)
        distances.sort(key=lambda x: x[1], reverse=False)
    #vdm
    elif (dataType == 1):
        for i in range(len(data)):
            distance = calcVDM(query, data[i], weighted, attributeCount, classList)
            distances.append((data[i], distance[0], distance[1]))
        distances.sort(key=lambda x: x[0], reverse=True)
        distances.sort(key=lambda x: x[1], reverse=False)
    
    #hvdm
    elif (dataType == 2):
        for i in range(len(data)):
            distance = calcHVDM(query, data[i], weighted, attributeCount, classList, typeList)
            distances.append((data[i], distance[0], distance[1]))
        distances.sort(key=lambda x: x[0], reverse=True)
        distances.sort(key=lambda x: x[1], reverse=False)
    
    classList = []
    for i in range(n):
        myClass = distances[i][0][-1]
        myWeight = distances[i][2]
        if (myClass not in classList):
            classes.append((myClass, myWeight))
            classList.append(myClass)
        else:
            for j in range(len(classes)):
                if (myClass == classes[j][0]):
                    classes[j] = list(classes[j])
                    classes[j][1] = classes[j][1] + myWeight
                    break
    
    classes.sort(key=lambda x: x[1], reverse=True)
    for i in range(len(distances)):
        print (distances[i])
    return classes

In [191]:
trainset = [[0, 0, 1, 'a'], [0, 1, 0, 'b'], [0,0,0, 'a'],[1,1,1,'b'],[1,1,0,'a']]
testInstance = [1, 0, 1]
k = 1
neighbors = knn(testInstance, 3, trainset, 0, 0, ['num','num','num','num','num'])
print(neighbors)
neighbors[0][0]

([1, 1, 1, 'b'], 1.0, 1)
([0, 0, 1, 'a'], 1.0, 1)
([1, 1, 0, 'a'], 1.4142135623730951, 1)
([0, 0, 0, 'a'], 1.4142135623730951, 1)
([0, 1, 0, 'b'], 1.7320508075688772, 1)
[['a', 2], ('b', 1)]


'a'

In [192]:
trainset = [[0, 0, 1, 'a'], [0, 1, 0, 'b'], [0,0,0, 'a'],[1,1,1,'b'],[1,1,0,'a']]
testInstance = [1, 0, 1]
k = 1
neighbors = knn(testInstance, 3, trainset, 1, 0, ['num','num','num','num','num'])
print(neighbors)
neighbors[0][0]

([1, 1, 1, 'b'], 1.0, 1.0)
([0, 0, 1, 'a'], 1.0, 1.0)
([1, 1, 0, 'a'], 1.4142135623730951, 0.4999999999999999)
([0, 0, 0, 'a'], 1.4142135623730951, 0.4999999999999999)
([0, 1, 0, 'b'], 1.7320508075688772, 0.33333333333333337)
[['a', 1.5], ('b', 1.0)]


'a'

In [193]:
trainset = [[0, 0, 1, 'a'], [0, 1, 0, 'b'], [0,0,0, 'a'],[1,1,1,'b'],[1,1,0,'a']]
testInstance = [1, 0, 1]
k = 1
neighbors = knn(testInstance, 3, trainset, 0, 1, ['num','num','num','num','num'])
print(neighbors)
neighbors[0][0]

([1, 1, 1, 'b'], 0.5773502691896258, 1)
([0, 0, 1, 'a'], 0.5773502691896258, 1)
([1, 1, 0, 'a'], 0.816496580927726, 1)
([0, 0, 0, 'a'], 0.816496580927726, 1)
([0, 1, 0, 'b'], 1.0, 1)
[['a', 2], ('b', 1)]


'a'

In [195]:
trainset = [[0, 0, 1, 'a'], [0, 1, 0, 'b'], [0,0,0, 'a'],[1,1,1,'b'],[1,1,0,'a']]
testInstance = [1, 0, 1]
k = 1
neighbors = knn(testInstance, 3, trainset, 0, 0, ['num','num','num','num','num'])
print(neighbors)
neighbors[0][0]

([1, 1, 1, 'b'], 1.0, 1)
([0, 0, 1, 'a'], 1.0, 1)
([1, 1, 0, 'a'], 1.4142135623730951, 1)
([0, 0, 0, 'a'], 1.4142135623730951, 1)
([0, 1, 0, 'b'], 1.7320508075688772, 1)
[['a', 2], ('b', 1)]


'a'

In [201]:
trainset = [[0, 0, 1, 'a'], [0, 1, 0, 'b'], [0,0,0, 'a'],[1,1,1,'b'],[1,1,0,'a']]
testInstance = [1, 0, 1]
k = 1
neighbors = knn(testInstance, 3, trainset, 0, 2, ['num','cat','cat','cat','num'])
print(neighbors)
neighbors[0][0]

([1, 1, 1, 'b'], 0.33333333333333337, 1)
([1, 1, 0, 'a'], 0.47140452079103173, 1)
([0, 0, 1, 'a'], 1.0, 1)
([0, 0, 0, 'a'], 1.0540925533894598, 1)
([0, 1, 0, 'b'], 1.1055415967851334, 1)
[['a', 2], ('b', 1)]


'a'

In [202]:
trainset = [[0, 0, 1, 'a'], [0, 1, 0, 'b'], [0,0,0, 'a'],[1,1,1,'b'],[1,1,0,'a']]
testInstance = [1, 0, 1]
k = 1
neighbors = knn(testInstance, 3, trainset, 1, 2, ['num','cat','cat','cat','num'])
print(neighbors)
neighbors[0][0]

([1, 1, 1, 'b'], 0.33333333333333337, 8.999999999999998)
([1, 1, 0, 'a'], 0.47140452079103173, 4.499999999999999)
([0, 0, 1, 'a'], 1.0, 1.0)
([0, 0, 0, 'a'], 1.0540925533894598, 0.8999999999999999)
([0, 1, 0, 'b'], 1.1055415967851334, 0.818181818181818)
[('b', 8.999999999999998), ['a', 5.499999999999999]]


'b'